In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

class HealthcareAIAgent:
    def __init__(self):
        self.scaler = StandardScaler()
        self.pca = PCA()
        self.clinical_knowledge_base = {}
        self.treatment_database = {}
        self.expert_weights = {}
        self.patient_data_processed = None
        
    def load_clinical_knowledge(self, expert_data: Dict):
        """
        Load clinical knowledge from medical experts
        
        Args:
            expert_data: Dictionary containing expert knowledge about treatments,
                        symptoms weights, and treatment efficacy
        """
        self.clinical_knowledge_base = expert_data
        print("✅ Clinical knowledge base loaded successfully")
        
    def collect_patient_data(self, patient_info: Dict) -> Dict:
        """
        Collect and structure patient data
        
        Args:
            patient_info: Raw patient data including demographics, symptoms, lab results
            
        Returns:
            Structured patient data dictionary
        """
        # Structure patient data
        structured_data = {
            'patient_id': patient_info.get('patient_id', 'Unknown'),
            'demographics': {
                'age': patient_info.get('age', 0),
                'gender': 1 if patient_info.get('gender', 'M') == 'M' else 0,
                'weight': patient_info.get('weight', 0),
                'height': patient_info.get('height', 0)
            },
            'symptoms': patient_info.get('symptoms', {}),
            'lab_results': patient_info.get('lab_results', {}),
            'medical_history': patient_info.get('medical_history', []),
            'allergies': patient_info.get('allergies', [])
        }
        
        print(f"📋 Patient data collected for ID: {structured_data['patient_id']}")
        return structured_data
    
    def regulate_patient_data_pca(self, patient_data_list: List[Dict]) -> np.ndarray:
        """
        Apply PCA to regulate and reduce dimensionality of patient data
        
        Args:
            patient_data_list: List of raw patient data dictionaries
            
        Returns:
            PCA-transformed patient data
        """
        # First, structure all patient data
        structured_patients = []
        for patient in patient_data_list:
            structured_data = self.collect_patient_data(patient)
            structured_patients.append(structured_data)
        
        # Get all unique symptom and lab result names for consistent feature ordering
        all_symptoms = set()
        all_lab_results = set()
        for patient in structured_patients:
            all_symptoms.update(patient['symptoms'].keys())
            all_lab_results.update(patient['lab_results'].keys())
        
        all_symptoms = sorted(list(all_symptoms))
        all_lab_results = sorted(list(all_lab_results))
        
        # Build feature matrix
        feature_matrix = []
        feature_names = ['age', 'gender', 'weight', 'height'] + all_symptoms + all_lab_results
        
        for patient in structured_patients:
            features = []
            
            # Demographics
            features.extend([
                patient['demographics']['age'],
                patient['demographics']['gender'],
                patient['demographics']['weight'],
                patient['demographics']['height']
            ])
            
            # Symptoms (fill missing with 0)
            for symptom in all_symptoms:
                features.append(patient['symptoms'].get(symptom, 0))
            
            # Lab results (fill missing with 0)
            for lab_test in all_lab_results:
                features.append(patient['lab_results'].get(lab_test, 0))
            
            feature_matrix.append(features)
        
        # Handle empty feature matrix
        if not feature_matrix:
            print("⚠️ No patient data to process")
            return np.array([])
        
        # Convert to numpy array and handle any remaining NaN values
        feature_matrix = np.array(feature_matrix, dtype=float)
        feature_matrix = np.nan_to_num(feature_matrix)  # Replace NaN with 0
        
        # Skip PCA if we have fewer than 2 samples or features
        if feature_matrix.shape[0] < 2 or feature_matrix.shape[1] < 2:
            print("⚠️ Insufficient data for PCA, returning original features")
            self.patient_data_processed = feature_matrix
            return feature_matrix
        
        # Standardize features
        feature_matrix_scaled = self.scaler.fit_transform(feature_matrix)
        
        # Apply PCA (adjust components based on available data)
        n_components = min(feature_matrix.shape[0] - 1, feature_matrix.shape[1], int(feature_matrix.shape[1] * 0.95))
        n_components = max(1, n_components)  # At least 1 component
        
        self.pca = PCA(n_components=n_components)
        pca_data = self.pca.fit_transform(feature_matrix_scaled)
        
        print(f"🔄 PCA applied: {feature_matrix.shape[1]} features → {pca_data.shape[1]} components")
        print(f"📊 Explained variance ratio: {self.pca.explained_variance_ratio_.sum():.3f}")
        
        self.patient_data_processed = pca_data
        return pca_data
    
    def get_possible_treatments(self, patient_condition: str) -> List[Dict]:
        """
        Retrieve all possible treatments for a patient condition
        
        Args:
            patient_condition: Primary diagnosis or condition
            
        Returns:
            List of possible treatments with their characteristics
        """
        # Sample treatment database (in real implementation, this would be comprehensive)
        treatment_db = {
            'hypertension': [
                {'name': 'ACE_Inhibitors', 'efficacy': 0.85, 'side_effects': 0.2, 'cost': 0.3, 'duration': 30},
                {'name': 'Beta_Blockers', 'efficacy': 0.80, 'side_effects': 0.3, 'cost': 0.25, 'duration': 30},
                {'name': 'Lifestyle_Changes', 'efficacy': 0.70, 'side_effects': 0.05, 'cost': 0.1, 'duration': 90},
                {'name': 'Surgery', 'efficacy': 0.95, 'side_effects': 0.4, 'cost': 0.9, 'duration': 1}
            ],
            'diabetes': [
                {'name': 'Metformin', 'efficacy': 0.82, 'side_effects': 0.25, 'cost': 0.2, 'duration': 365},
                {'name': 'Insulin', 'efficacy': 0.90, 'side_effects': 0.3, 'cost': 0.6, 'duration': 365},
                {'name': 'Diet_Exercise', 'efficacy': 0.65, 'side_effects': 0.05, 'cost': 0.1, 'duration': 365},
                {'name': 'Surgery', 'efficacy': 0.88, 'side_effects': 0.35, 'cost': 0.85, 'duration': 1}
            ]
        }
        
        treatments = treatment_db.get(patient_condition.lower(), [])
        print(f"🔍 Found {len(treatments)} possible treatments for {patient_condition}")
        
        self.treatment_database[patient_condition] = treatments
        return treatments
    
    def calculate_criteria_weights(self, patient_profile: Dict) -> Dict[str, float]:
        """
        Calculate weights for different criteria based on patient profile and expert knowledge
        
        Args:
            patient_profile: Patient's demographic and clinical profile
            
        Returns:
            Dictionary of criteria weights
        """
        # Base weights from clinical expertise
        base_weights = {
            'efficacy': 0.4,
            'side_effects': 0.25,
            'cost': 0.2,
            'duration': 0.15
        }
        
        # Adjust weights based on patient factors
        age = patient_profile['demographics']['age']
        
        # Elderly patients: prioritize safety over efficacy
        if age > 70:
            base_weights['efficacy'] = 0.3
            base_weights['side_effects'] = 0.4
        
        # Young patients: prioritize efficacy and duration
        elif age < 30:
            base_weights['efficacy'] = 0.5
            base_weights['duration'] = 0.2
            base_weights['side_effects'] = 0.15
        
        # Normalize weights
        total_weight = sum(base_weights.values())
        normalized_weights = {k: v/total_weight for k, v in base_weights.items()}
        
        print("⚖️ Criteria weights calculated:", normalized_weights)
        return normalized_weights
    
    def topsis_analysis(self, treatments: List[Dict], weights: Dict[str, float]) -> List[Tuple[str, float]]:
        """
        Apply TOPSIS method for treatment ranking
        
        Args:
            treatments: List of treatment options
            weights: Criteria weights
            
        Returns:
            Ranked treatments with TOPSIS scores
        """
        if not treatments:
            return []
        
        # Create decision matrix
        criteria_names = list(weights.keys())
        decision_matrix = np.array([[t[criterion] for criterion in criteria_names] for t in treatments])
        
        # Normalize decision matrix
        normalized_matrix = decision_matrix / np.sqrt(np.sum(decision_matrix**2, axis=0))
        
        # Apply weights
        weighted_matrix = normalized_matrix * np.array(list(weights.values()))
        
        # Determine ideal and negative ideal solutions
        # For efficacy: higher is better (beneficial)
        # For side_effects, cost, duration: lower is better (non-beneficial)
        beneficial = [True, False, False, False]  # efficacy=beneficial, others=non-beneficial
        
        ideal_solution = np.zeros(len(criteria_names))
        negative_ideal_solution = np.zeros(len(criteria_names))
        
        for i, is_beneficial in enumerate(beneficial):
            if is_beneficial:
                ideal_solution[i] = np.max(weighted_matrix[:, i])
                negative_ideal_solution[i] = np.min(weighted_matrix[:, i])
            else:
                ideal_solution[i] = np.min(weighted_matrix[:, i])
                negative_ideal_solution[i] = np.max(weighted_matrix[:, i])
        
        # Calculate distances to ideal solutions
        distance_to_ideal = np.sqrt(np.sum((weighted_matrix - ideal_solution)**2, axis=1))
        distance_to_negative_ideal = np.sqrt(np.sum((weighted_matrix - negative_ideal_solution)**2, axis=1))
        
        # Calculate TOPSIS scores
        topsis_scores = distance_to_negative_ideal / (distance_to_ideal + distance_to_negative_ideal)
        
        # Rank treatments
        treatment_scores = [(treatments[i]['name'], topsis_scores[i]) for i in range(len(treatments))]
        treatment_scores.sort(key=lambda x: x[1], reverse=True)
        
        print("📊 TOPSIS analysis completed")
        return treatment_scores
    
    def waspas_analysis(self, treatments: List[Dict], weights: Dict[str, float], lambda_param: float = 0.5) -> List[Tuple[str, float]]:
        """
        Apply WASPAS method for robust treatment ranking
        
        Args:
            treatments: List of treatment options
            weights: Criteria weights
            lambda_param: Balance between WSM and WPM (0.5 = equal weight)
            
        Returns:
            Ranked treatments with WASPAS scores
        """
        if not treatments:
            return []
        
        criteria_names = list(weights.keys())
        decision_matrix = np.array([[t[criterion] for criterion in criteria_names] for t in treatments])
        
        # Normalize decision matrix (min-max normalization)
        min_vals = np.min(decision_matrix, axis=0)
        max_vals = np.max(decision_matrix, axis=0)
        
        normalized_matrix = np.zeros_like(decision_matrix)
        beneficial = [True, False, False, False]  # efficacy=beneficial, others=non-beneficial
        
        for i, is_beneficial in enumerate(beneficial):
            if max_vals[i] != min_vals[i]:  # Avoid division by zero
                if is_beneficial:
                    normalized_matrix[:, i] = (decision_matrix[:, i] - min_vals[i]) / (max_vals[i] - min_vals[i])
                else:
                    normalized_matrix[:, i] = (max_vals[i] - decision_matrix[:, i]) / (max_vals[i] - min_vals[i])
            else:
                normalized_matrix[:, i] = 1.0  # If all values are same, assign 1
        
        # WSM (Weighted Sum Model)
        wsm_scores = np.sum(normalized_matrix * np.array(list(weights.values())), axis=1)
        
        # WPM (Weighted Product Model)
        # Avoid zero values for logarithm
        normalized_matrix_wpm = np.maximum(normalized_matrix, 1e-10)
        wpm_scores = np.prod(normalized_matrix_wpm ** np.array(list(weights.values())), axis=1)
        
        # Normalize scores to [0,1] range
        wsm_scores = wsm_scores / np.max(wsm_scores) if np.max(wsm_scores) > 0 else wsm_scores
        wpm_scores = wpm_scores / np.max(wpm_scores) if np.max(wpm_scores) > 0 else wpm_scores
        
        # WASPAS scores (combination of WSM and WPM)
        waspas_scores = lambda_param * wsm_scores + (1 - lambda_param) * wpm_scores
        
        # Rank treatments
        treatment_scores = [(treatments[i]['name'], waspas_scores[i]) for i in range(len(treatments))]
        treatment_scores.sort(key=lambda x: x[1], reverse=True)
        
        print("🔧 WASPAS analysis completed")
        return treatment_scores
    
    def recommend_treatment(self, patient_data: Dict, condition: str) -> Dict:
        """
        Main method to recommend optimal treatment for a patient
        
        Args:
            patient_data: Patient information
            condition: Primary medical condition
            
        Returns:
            Treatment recommendation with analysis results
        """
        print(f"\n🏥 STARTING TREATMENT RECOMMENDATION FOR PATIENT: {patient_data.get('patient_id', 'Unknown')}")
        print("="*80)
        
        # Step 1: Collect and process patient data
        structured_data = self.collect_patient_data(patient_data)
        
        # Step 2: Get possible treatments
        treatments = self.get_possible_treatments(condition)
        if not treatments:
            return {"error": f"No treatments found for condition: {condition}"}
        
        # Step 3: Calculate criteria weights based on patient profile
        weights = self.calculate_criteria_weights(structured_data)
        
        # Step 4: Apply TOPSIS analysis
        topsis_results = self.topsis_analysis(treatments, weights)
        
        # Step 5: Apply WASPAS for robust validation
        waspas_results = self.waspas_analysis(treatments, weights)
        
        # Step 6: Combine results for final recommendation
        final_recommendation = self.combine_rankings(topsis_results, waspas_results)
        
        # Prepare comprehensive report
        recommendation_report = {
            'patient_id': structured_data['patient_id'],
            'condition': condition,
            'recommended_treatment': final_recommendation['treatment'],
            'confidence_score': final_recommendation['score'],
            'criteria_weights': weights,
            'topsis_ranking': topsis_results,
            'waspas_ranking': waspas_results,
            'analysis_summary': self.generate_analysis_summary(final_recommendation, weights)
        }
        
        print(f"\n✅ RECOMMENDATION COMPLETE")
        print(f"🎯 Recommended Treatment: {final_recommendation['treatment']}")
        print(f"📈 Confidence Score: {final_recommendation['score']:.3f}")
        print("="*80)
        
        return recommendation_report
    
    def combine_rankings(self, topsis_results: List[Tuple[str, float]], 
                        waspas_results: List[Tuple[str, float]]) -> Dict:
        """
        Combine TOPSIS and WASPAS rankings for final recommendation
        
        Args:
            topsis_results: Ranked treatments from TOPSIS
            waspas_results: Ranked treatments from WASPAS
            
        Returns:
            Final recommendation with combined score
        """
        # Convert to dictionaries for easier access
        topsis_scores = {name: score for name, score in topsis_results}
        waspas_scores = {name: score for name, score in waspas_results}
        
        # Calculate combined scores (equal weight to both methods)
        combined_scores = {}
        for treatment in topsis_scores:
            combined_score = 0.5 * topsis_scores[treatment] + 0.5 * waspas_scores.get(treatment, 0)
            combined_scores[treatment] = combined_score
        
        # Find best treatment
        best_treatment = max(combined_scores, key=combined_scores.get)
        best_score = combined_scores[best_treatment]
        
        return {
            'treatment': best_treatment,
            'score': best_score,
            'all_scores': combined_scores
        }
    
    def generate_analysis_summary(self, recommendation: Dict, weights: Dict) -> str:
        """
        Generate human-readable analysis summary
        
        Args:
            recommendation: Final treatment recommendation
            weights: Criteria weights used in analysis
            
        Returns:
            Summary string explaining the recommendation
        """
        treatment = recommendation['treatment']
        score = recommendation['score']
        
        # Identify primary decision factors
        primary_factor = max(weights, key=weights.get)
        
        summary = f"""
        TREATMENT ANALYSIS SUMMARY:
        
        Recommended Treatment: {treatment}
        Overall Confidence: {score:.1%}
        
        Key Decision Factors:
        - Primary consideration: {primary_factor} (weight: {weights[primary_factor]:.1%})
        - This recommendation balances all clinical criteria effectively
        - Both TOPSIS and WASPAS methods support this choice
        
        Clinical Rationale:
        The selected treatment optimizes the patient-specific criteria weights
        while considering efficacy, safety, cost, and treatment duration.
        """
        
        return summary.strip()
    
    def visualize_analysis(self, recommendation_report: Dict):
        """
        Create visualizations of the treatment analysis
        
        Args:
            recommendation_report: Complete recommendation report
        """
        plt.figure(figsize=(15, 10))
        
        # Subplot 1: Criteria Weights
        plt.subplot(2, 3, 1)
        weights = recommendation_report['criteria_weights']
        plt.pie(weights.values(), labels=weights.keys(), autopct='%1.1f%%')
        plt.title('Criteria Weights for Patient')
        
        # Subplot 2: TOPSIS Scores
        plt.subplot(2, 3, 2)
        topsis_data = recommendation_report['topsis_ranking'][:5]  # Top 5
        treatments = [t[0] for t in topsis_data]
        scores = [t[1] for t in topsis_data]
        plt.barh(treatments, scores)
        plt.title('TOPSIS Ranking')
        plt.xlabel('Score')
        
        # Subplot 3: WASPAS Scores
        plt.subplot(2, 3, 3)
        waspas_data = recommendation_report['waspas_ranking'][:5]  # Top 5
        treatments = [t[0] for t in waspas_data]
        scores = [t[1] for t in waspas_data]
        plt.barh(treatments, scores)
        plt.title('WASPAS Ranking')
        plt.xlabel('Score')
        
        plt.tight_layout()
        plt.show()

# Example Usage and Workflow
def demonstrate_healthcare_ai():
    """
    Demonstrate the complete workflow of the Healthcare AI Agent
    """
    print("🚀 HEALTHCARE AI AGENT DEMONSTRATION")
    print("="*50)
    
    # Initialize the AI agent
    agent = HealthcareAIAgent()
    
    # Load expert clinical knowledge (this would typically come from medical databases)
    expert_knowledge = {
        'treatment_protocols': {
            'hypertension': ['medication', 'lifestyle', 'surgery'],
            'diabetes': ['medication', 'diet', 'insulin', 'surgery']
        },
        'efficacy_data': {
            'ACE_Inhibitors': 0.85,
            'Beta_Blockers': 0.80,
            'Lifestyle_Changes': 0.70
        }
    }
    agent.load_clinical_knowledge(expert_knowledge)
    
    # Sample patient data input
    sample_patients = [
        {
            'patient_id': 'P001',
            'age': 65,
            'gender': 'M',
            'weight': 80,
            'height': 175,
            'symptoms': {
                'blood_pressure_systolic': 160,
                'blood_pressure_diastolic': 95,
                'chest_pain': 3,  # Scale 1-5
                'headache': 2
            },
            'lab_results': {
                'cholesterol': 220,
                'glucose': 110,
                'creatinine': 1.1
            },
            'medical_history': ['smoking', 'family_history_heart_disease'],
            'allergies': ['penicillin']
        },
        {
            'patient_id': 'P002',
            'age': 45,
            'gender': 'F',
            'weight': 65,
            'height': 160,
            'symptoms': {
                'blood_pressure_systolic': 145,
                'blood_pressure_diastolic': 90,
                'fatigue': 4,
                'dizziness': 2
            },
            'lab_results': {
                'cholesterol': 180,
                'glucose': 95,
                'creatinine': 0.9
            },
            'medical_history': ['diabetes_family'],
            'allergies': []
        }
    ]
    
    # Process multiple patients for PCA analysis
    agent.regulate_patient_data_pca(sample_patients)
    
    # Generate treatment recommendation for first patient
    recommendation = agent.recommend_treatment(sample_patients[0], 'hypertension')
    
    # Display results
    print("\n📋 DETAILED RECOMMENDATION REPORT:")
    print("-" * 50)
    print(f"Patient ID: {recommendation['patient_id']}")
    print(f"Condition: {recommendation['condition']}")
    print(f"Recommended Treatment: {recommendation['recommended_treatment']}")
    print(f"Confidence Score: {recommendation['confidence_score']:.3f}")
    
    print("\n📊 RANKING COMPARISON:")
    print("TOPSIS Results:")
    for i, (treatment, score) in enumerate(recommendation['topsis_ranking'][:3], 1):
        print(f"  {i}. {treatment}: {score:.3f}")
    
    print("WASPAS Results:")
    for i, (treatment, score) in enumerate(recommendation['waspas_ranking'][:3], 1):
        print(f"  {i}. {treatment}: {score:.3f}")
    
    print("\n💡 ANALYSIS SUMMARY:")
    print(recommendation['analysis_summary'])
    
    return agent, recommendation

# Input Workflow Guide
def input_workflow_guide():
    """
    Provide detailed guide for input workflow and weight regulation
    """
    guide = """
    
    📝 INPUT WORKFLOW GUIDE
    =====================
    
    1. PATIENT DATA COLLECTION:
       - Demographics: age, gender, weight, height
       - Symptoms: numerical severity scores (1-5 scale)
       - Lab Results: actual test values
       - Medical History: list of relevant conditions
       - Allergies: list of known allergies
    
    2. WEIGHT REGULATION METHODS:
    
       A. Symptom Weights:
          - Based on clinical severity and diagnostic relevance
          - Expert-defined severity multipliers
          - Patient-specific risk factors
    
       B. Treatment Weights (Criteria):
          - Efficacy: Clinical success rate (0-1)
          - Side Effects: Risk probability (0-1, lower is better)
          - Cost: Relative cost (0-1, lower is better)
          - Duration: Treatment length in days
    
       C. Patient-Specific Adjustments:
          - Age-based weight modifications
          - Comorbidity considerations
          - Risk tolerance factors
    
    3. INPUT EXAMPLES:
    
       Patient Input Format:
       {
           'patient_id': 'unique_identifier',
           'age': 65,
           'gender': 'M' or 'F',
           'weight': 80,  # kg
           'height': 175,  # cm
           'symptoms': {
               'symptom_name': severity_score,  # 1-5 scale
               'blood_pressure_systolic': 160,
               'pain_level': 3
           },
           'lab_results': {
               'test_name': actual_value,
               'cholesterol': 220,
               'glucose': 110
           },
           'medical_history': ['condition1', 'condition2'],
           'allergies': ['drug1', 'drug2']
       }
    
    4. WEIGHT CALCULATION LOGIC:
    
       - Base weights from clinical guidelines
       - Patient age adjustments:
         * Elderly (>70): Safety priority
         * Young (<30): Efficacy priority
       - Comorbidity adjustments
       - Cost sensitivity based on socioeconomic factors
    
    5. OUTPUT INTERPRETATION:
    
       - TOPSIS Score: Distance-based ranking (higher = better)
       - WASPAS Score: Combined WSM+WPM (higher = better)
       - Final Score: Weighted combination of both methods
       - Confidence: Overall reliability of recommendation
    
    """
    print(guide)

if __name__ == "__main__":
    # Run the demonstration
    agent, results = demonstrate_healthcare_ai()
    
    # Show input workflow guide
    input_workflow_guide()
    
    print("\n🎉 Healthcare AI Agent demonstration completed!")
    print("Ready for real-world clinical decision support!")

🚀 HEALTHCARE AI AGENT DEMONSTRATION
✅ Clinical knowledge base loaded successfully
📋 Patient data collected for ID: P001
📋 Patient data collected for ID: P002
🔄 PCA applied: 13 features → 1 components
📊 Explained variance ratio: 1.000

🏥 STARTING TREATMENT RECOMMENDATION FOR PATIENT: P001
📋 Patient data collected for ID: P001
🔍 Found 4 possible treatments for hypertension
⚖️ Criteria weights calculated: {'efficacy': 0.4, 'side_effects': 0.25, 'cost': 0.2, 'duration': 0.15}
📊 TOPSIS analysis completed
🔧 WASPAS analysis completed

✅ RECOMMENDATION COMPLETE
🎯 Recommended Treatment: ACE_Inhibitors
📈 Confidence Score: 0.829

📋 DETAILED RECOMMENDATION REPORT:
--------------------------------------------------
Patient ID: P001
Condition: hypertension
Recommended Treatment: ACE_Inhibitors
Confidence Score: 0.829

📊 RANKING COMPARISON:
TOPSIS Results:
  1. ACE_Inhibitors: 0.657
  2. Lifestyle_Changes: 0.609
  3. Beta_Blockers: 0.560
WASPAS Results:
  1. ACE_Inhibitors: 1.000
  2. Beta_Blockers: 

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

class HealthcareAIAgent:
    def __init__(self):
        self.scaler = StandardScaler()
        self.pca = PCA()
        self.clinical_knowledge_base = {}
        self.treatment_database = {}
        self.patient_data_processed = None

    def load_clinical_knowledge(self, expert_data: Dict):
        self.clinical_knowledge_base = expert_data
        print("✅ Clinical knowledge base loaded successfully")

    def collect_patient_data(self, patient_info: Dict) -> Dict:
        structured_data = {
            'patient_id': patient_info.get('patient_id', 'Unknown'),
            'demographics': {
                'age': patient_info.get('age', 0),
                'gender': 1 if patient_info.get('gender', 'M') == 'M' else 0,
                'weight': patient_info.get('weight', 0),
                'height': patient_info.get('height', 0)
            },
            'symptoms': patient_info.get('symptoms', {}),
            'lab_results': patient_info.get('lab_results', {}),
            'medical_history': patient_info.get('medical_history', []),
            'allergies': patient_info.get('allergies', [])
        }
        print(f"📋 Patient data collected for ID: {structured_data['patient_id']}")
        return structured_data

    def get_possible_treatments(self, patient_condition: str) -> List[Dict]:
        treatment_db = {
            'hypertension': [
                {'name': 'ACE_Inhibitors', 'efficacy': 0.85, 'side_effects': 0.2, 'cost': 0.3, 'duration': 30},
                {'name': 'Beta_Blockers', 'efficacy': 0.80, 'side_effects': 0.3, 'cost': 0.25, 'duration': 30},
                {'name': 'Lifestyle_Changes', 'efficacy': 0.70, 'side_effects': 0.05, 'cost': 0.1, 'duration': 90},
                {'name': 'Surgery', 'efficacy': 0.95, 'side_effects': 0.4, 'cost': 0.9, 'duration': 1}
            ]
        }
        treatments = treatment_db.get(patient_condition.lower(), [])
        print(f"🔍 Found {len(treatments)} possible treatments for {patient_condition}")
        self.treatment_database[patient_condition] = treatments
        return treatments

    def calculate_criteria_weights(self, patient_profile: Dict) -> Dict[str, float]:
        base_weights = {'efficacy': 0.4, 'side_effects': 0.25, 'cost': 0.2, 'duration': 0.15}
        age = patient_profile['demographics']['age']
        if age > 70:
            base_weights['efficacy'] = 0.3
            base_weights['side_effects'] = 0.4
        elif age < 30:
            base_weights['efficacy'] = 0.5
            base_weights['duration'] = 0.2
            base_weights['side_effects'] = 0.15
        total = sum(base_weights.values())
        weights = {k: v / total for k, v in base_weights.items()}
        print("⚖️ Criteria weights calculated:", weights)
        return weights

    def topsis_analysis(self, treatments: List[Dict], weights: Dict[str, float]) -> List[Tuple[str, float]]:
        criteria = list(weights.keys())
        decision_matrix = np.array([[t[c] for c in criteria] for t in treatments])
        normalized = decision_matrix / np.sqrt((decision_matrix ** 2).sum(axis=0))
        weighted = normalized * np.array(list(weights.values()))
        beneficial = [True, False, False, False]
        ideal, nadir = np.zeros(len(criteria)), np.zeros(len(criteria))
        for i, ben in enumerate(beneficial):
            if ben:
                ideal[i], nadir[i] = weighted[:, i].max(), weighted[:, i].min()
            else:
                ideal[i], nadir[i] = weighted[:, i].min(), weighted[:, i].max()
        dist_ideal = np.sqrt(((weighted - ideal) ** 2).sum(axis=1))
        dist_nadir = np.sqrt(((weighted - nadir) ** 2).sum(axis=1))
        scores = dist_nadir / (dist_ideal + dist_nadir)
        results = sorted([(t['name'], scores[i]) for i, t in enumerate(treatments)], key=lambda x: x[1], reverse=True)
        print("📊 TOPSIS analysis completed")
        return results

    def waspas_analysis(self, treatments: List[Dict], weights: Dict[str, float], lam: float = 0.5) -> List[Tuple[str, float]]:
        criteria = list(weights.keys())
        decision_matrix = np.array([[t[c] for c in criteria] for t in treatments])
        min_vals, max_vals = decision_matrix.min(axis=0), decision_matrix.max(axis=0)
        beneficial = [True, False, False, False]
        normalized = np.zeros_like(decision_matrix)
        for i, ben in enumerate(beneficial):
            if max_vals[i] != min_vals[i]:
                if ben:
                    normalized[:, i] = (decision_matrix[:, i] - min_vals[i]) / (max_vals[i] - min_vals[i])
                else:
                    normalized[:, i] = (max_vals[i] - decision_matrix[:, i]) / (max_vals[i] - min_vals[i])
            else:
                normalized[:, i] = 1.0
        wsm = (normalized * np.array(list(weights.values()))).sum(axis=1)
        normalized_wpm = np.maximum(normalized, 1e-10)
        wpm = np.prod(normalized_wpm ** np.array(list(weights.values())), axis=1)
        wsm /= wsm.max() if wsm.max() > 0 else 1
        wpm /= wpm.max() if wpm.max() > 0 else 1
        scores = lam * wsm + (1 - lam) * wpm
        results = sorted([(t['name'], scores[i]) for i, t in enumerate(treatments)], key=lambda x: x[1], reverse=True)
        print("🔧 WASPAS analysis completed")
        return results

    def combine_rankings(self, topsis_results, waspas_results):
        topsis_scores = dict(topsis_results)
        waspas_scores = dict(waspas_results)
        combined = {t: 0.5 * topsis_scores[t] + 0.5 * waspas_scores[t] for t in topsis_scores}
        best = max(combined, key=combined.get)
        return {'treatment': best, 'score': combined[best], 'all_scores': combined}

    def generate_analysis_summary(self, recommendation: Dict, weights: Dict) -> str:
        primary = max(weights, key=weights.get)
        return (
            f"TREATMENT ANALYSIS SUMMARY:\n"
            f"Recommended Treatment: {recommendation['treatment']}\n"
            f"Overall Confidence: {recommendation['score']:.1%}\n"
            f"Primary consideration: {primary} (weight: {weights[primary]:.1%})\n"
            f"Both TOPSIS and WASPAS agree with this choice."
        )

    def recommend_treatment_basic(self, patient_data: dict, condition: str) -> dict:
        structured = self.collect_patient_data(patient_data)
        treatments = self.get_possible_treatments(condition)
        weights = self.calculate_criteria_weights(structured)
        topsis_res = self.topsis_analysis(treatments, weights)
        waspas_res = self.waspas_analysis(treatments, weights)
        final = self.combine_rankings(topsis_res, waspas_res)
        return {
            'patient_id': structured['patient_id'],
            'condition': condition,
            'recommended_treatment': final['treatment'],
            'confidence_score': final['score']
        }

    def recommend_treatment_details(self, patient_data: dict, condition: str) -> dict:
        structured = self.collect_patient_data(patient_data)
        treatments = self.get_possible_treatments(condition)
        weights = self.calculate_criteria_weights(structured)
        topsis_res = self.topsis_analysis(treatments, weights)
        waspas_res = self.waspas_analysis(treatments, weights)
        final = self.combine_rankings(topsis_res, waspas_res)
        return {
            'patient_id': structured['patient_id'],
            'condition': condition,
            'recommended_treatment': final['treatment'],
            'confidence_score': final['score'],
            'criteria_weights': weights,
            'topsis_ranking': topsis_res,
            'waspas_ranking': waspas_res,
            'analysis_summary': self.generate_analysis_summary(final, weights)
        }


# -------------------
# Menu Loop
# -------------------
if __name__ == "__main__":
    expert_knowledge = {
        'treatment_protocols': {
            'hypertension': ['medication', 'lifestyle', 'surgery']
        },
        'efficacy_data': {
            'ACE_Inhibitors': 0.85,
            'Beta_Blockers': 0.80,
            'Lifestyle_Changes': 0.70
        }
    }

    patient = {
        'patient_id': 'P001',
        'age': 65,
        'gender': 'M',
        'weight': 80,
        'height': 175,
        'symptoms': {
            'blood_pressure_systolic': 160,
            'blood_pressure_diastolic': 95
        },
        'lab_results': {
            'cholesterol': 220,
            'glucose': 110
        },
        'medical_history': ['smoking'],
        'allergies': []
    }

    agent = HealthcareAIAgent()
    agent.load_clinical_knowledge(expert_knowledge)

    while True:
        print("\n========== MENU ==========")
        print("1. Basic Treatment Recommendation (minimal output)")
        print("2. Detailed Recommendation Report (full analysis)")
        print("3. Exit")
        choice = input("Enter your choice (1, 2 or 3): ").strip()

        if choice == '1':
            result = agent.recommend_treatment_basic(patient, 'hypertension')
            print("\n=== BASIC TREATMENT RECOMMENDATION ===")
            print(f"Patient ID: {result['patient_id']}")
            print(f"Condition: {result['condition']}")
            print(f"Recommended Treatment: {result['recommended_treatment']}")
            print(f"Confidence Score: {result['confidence_score']:.3f}")

        elif choice == '2':
            result = agent.recommend_treatment_details(patient, 'hypertension')
            print("\n=== DETAILED RECOMMENDATION REPORT ===")
            print(f"Patient ID: {result['patient_id']}")
            print(f"Condition: {result['condition']}")
            print(f"Recommended Treatment: {result['recommended_treatment']}")
            print(f"Confidence Score: {result['confidence_score']:.3f}")
            print("\nTOPSIS Ranking:")
            for t, s in result['topsis_ranking']:
                print(f"  {t}: {s:.3f}")
            print("\nWASPAS Ranking:")
            for t, s in result['waspas_ranking']:
                print(f"  {t}: {s:.3f}")
            print("\nAnalysis Summary:")
            print(result['analysis_summary'])

        elif choice == '3':
            print("✅ Exiting program. Goodbye!")
            break
        else:
            print("❌ Invalid choice. Please select 1, 2, or 3.")


✅ Clinical knowledge base loaded successfully

========== MENU ==========
1. Basic Treatment Recommendation (minimal output)
2. Detailed Recommendation Report (full analysis)
3. Exit


Enter your choice (1, 2 or 3):  1


📋 Patient data collected for ID: P001
🔍 Found 4 possible treatments for hypertension
⚖️ Criteria weights calculated: {'efficacy': 0.4, 'side_effects': 0.25, 'cost': 0.2, 'duration': 0.15}
📊 TOPSIS analysis completed
🔧 WASPAS analysis completed

=== BASIC TREATMENT RECOMMENDATION ===
Patient ID: P001
Condition: hypertension
Recommended Treatment: ACE_Inhibitors
Confidence Score: 0.829

========== MENU ==========
1. Basic Treatment Recommendation (minimal output)
2. Detailed Recommendation Report (full analysis)
3. Exit


Enter your choice (1, 2 or 3):  2


📋 Patient data collected for ID: P001
🔍 Found 4 possible treatments for hypertension
⚖️ Criteria weights calculated: {'efficacy': 0.4, 'side_effects': 0.25, 'cost': 0.2, 'duration': 0.15}
📊 TOPSIS analysis completed
🔧 WASPAS analysis completed

=== DETAILED RECOMMENDATION REPORT ===
Patient ID: P001
Condition: hypertension
Recommended Treatment: ACE_Inhibitors
Confidence Score: 0.829

TOPSIS Ranking:
  ACE_Inhibitors: 0.657
  Lifestyle_Changes: 0.609
  Beta_Blockers: 0.560
  Surgery: 0.391

WASPAS Ranking:
  ACE_Inhibitors: 1.000
  Beta_Blockers: 0.754
  Surgery: 0.434
  Lifestyle_Changes: 0.355

Analysis Summary:
TREATMENT ANALYSIS SUMMARY:
Recommended Treatment: ACE_Inhibitors
Overall Confidence: 82.9%
Primary consideration: efficacy (weight: 40.0%)
Both TOPSIS and WASPAS agree with this choice.

========== MENU ==========
1. Basic Treatment Recommendation (minimal output)
2. Detailed Recommendation Report (full analysis)
3. Exit
